# ONLY RUN THE NEXT CELL ONCE!

In [10]:

from weis_workshop import *

if IN_COLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install() # expect a kernel restart
    from google.colab import drive
    drive.mount('/content/drive')
    !git clone -b weis-workshop  https://github.com/mayankchetan/weis
    !mamba env update -n base -f weis/environment.yml
    !mamba install -y petsc4py=3.22.2 mpi4py pyoptsparse
    %cd weis
    !pip install .
    %cd ..
    !git clone https://github.com/mayankchetan/weisWorkshop-Private

else:
    try:
        import weis
    except ImportError:
        raise ImportError("Please install the weis package from https://github.com/mayankchetan/weis in the environment you are using to run this notebook.")
    
    Warning("Make sure you have cloned 'https://github.com/mayankchetan/weisWorkshop-Private' repository and are running the notebook from the root directory of the repository.")




Running in local environment


# Introduction
 > Write some info

# Objectives
You are assigned to design and optimize a 20MW Floating wind turbine with the following constriants.
1. 
2. 
3. 

# The Approach 
1. Start with the IEA 22MW folating turbine. Establish a baseline 
2. Modify the rotor, try aero, structural and aero + structural optimization
3. Optimize the controller to the updated rotor
4. Optimize the substructure uisng RAFT
5. Continue optimization using OpenFAST & OpenFAST+Controller
6. Conduct a full DLC analysis

In [12]:
# Importing nesscary libraries.
import numpy as np
import matplotlib.pyplot as plt
from weis import weis_main
from wisdem import run_wisdem
import os
import copy
import pandas as pd

# workshop repository folder
WORKSHOP_REPO = '/projects/weis/mchetan/weis-workshop/workshopFiles'

## Stage 0: Establishing a baseline

We begin with establishing a baseline using an exisitng wind turbine model. Since we are desgining an 20MW model, lets start with the IEA 22MW turbine. 

In [ ]:
# Setting up file paths
stageFolder = "stage-0-baseline"

fname_wt_input = os.path.join(WORKSHOP_REPO, "source", "IEA-22-280-RWT_Floater.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, "stage-0-baseline_modeling.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, "stage-0-baseline_analysis.yaml")

#

## Lets Open the input files and take a Look at them.
### Google Colab Users:
1. Go to the file browser on the left
2. Navigate to the cloned repository
3. Open the YAML files to examine the configuration

## Turibne geometry file
This file defines the trubine using the WindIO format. Currently we are using the v2.0 of WindIO. Lets take a look at the high level parts of the file.

```yaml
name: IEA 22MW Offshore Wind Turbine in Floating Configuration
assembly:
    turbine_class: I
    turbulence_class: B
    drivetrain: direct_drive
    rotor_orientation: Upwind
    number_of_blades: 3
    hub_height: 170.0
    rotor_diameter: 284.0
    rated_power: 22000000.0
    lifetime: 25.0
components:
  blade:
    outer_shape_bem:
      reference_axis:
        x: { grid: [...], values: [...] }
        y: { grid: [...], values: [...] }
        z: { grid: [...], values: [...] }
      chord: { grid: [...], values: [...] }
      twist: { grid: [...], values: [...] }
    internal_structure_2d_fem:
      reference_axis: [...]
      layers: [...]
      webs: [...]
  hub:
    diameter: 7.94  # m
    cone_angle: -2.5  # degrees
    
  nacelle:
    drivetrain: [...]
    transformer: [...]
    
  tower:
    outer_shape_bem: [...]
    internal_structure_2d_fem: [...]
    
  mooring:
    nodes: [...]
    lines: [...]
airfoils: [...]
materials: [...]
controls: [...]
environment: [...]
bos: [...]
costs: [...]
```


## analysis and modeling options
These files define the analysis and modeling options for the WISDEM/WEIS

### Analysis options
```yaml
general:
    folder_output: outputs/
    fname_output: stage-0-baseline
```
### Modeling options
```yaml
General:
    verbosity: False

WISDEM:
    RotorSE:
        flag: True
    TowerSE:
        flag: True
    DriveSE:
        flag: True
    FloatingSE:
        flag: True
    BOS:
        flag: True

OpenFAST: # Options for WEIS fidelity level 3 = nonlinear time domain
    flag: False

RAFT:
    flag: False

ROSCO:
    flag: True
    tuning_yaml: ../source/iea22_rosco.yaml
```


In [9]:
wt_opt, modeling_options, opt_options = weis_main(fname_wt_input, 
                                                 fname_modeling_options, 
                                                 fname_analysis_options,
                                                 test_run=False
                                                 )

Accessing Draft4Validator.resolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization./kfs2/projects/weis/mchetan/weis-workshop/env/weis-workshop-script1/lib/python3.12/site-packages/openmdao/core/driver.py:769: OpenMDAOWarning:Driver: No matches for pattern '*_df' in recording_options['excludes'].
/kfs2/projects/weis/mchetan/weis-workshop/env/weis-workshop-script1/lib/python3.12/site-packages/openmdao/recorders/sqlite_recorder.py:231: UserWarning:The existing case recorder file, /projects/weis/mchetan/weis-workshop/workshopFiles/stage-0-baseline/outputs/log_opt.sql, is being overwritten.
/kfs2/projects/weis/mchetan/weis-workshop/env/weis-workshop-script1/lib/python3.12/site-packages/openmdao/visualization/n2_viewer/n2_viewer.py:114: OpenMDAOWarning:All-NaN slice encountered



wisdem.wt.wt_rna


NL: NLBGS 1 ; 5.83474204e+11 1


NL: NLBGS 2 ; 28963702.2 4.96400732e-05


NL: NLBGS 3 ; 102078.295 1.74949114e-07


NL: NLBGS 4 ; 236.774411 4.05800993e-10
NL: NLBGS Converged


ORBIT library intialized at '/kfs2/projects/weis/mchetan/weis-workshop/env/weis-workshop-script1/lib/python3.12/site-packages/library'
Cp-Ct-Cq surfaces completed at 5 %
Cp-Ct-Cq surfaces completed at 10 %
Cp-Ct-Cq surfaces completed at 15 %
Cp-Ct-Cq surfaces completed at 20 %
Cp-Ct-Cq surfaces completed at 25 %
Cp-Ct-Cq surfaces completed at 30 %
Cp-Ct-Cq surfaces completed at 35 %
Cp-Ct-Cq surfaces completed at 40 %
Cp-Ct-Cq surfaces completed at 45 %
Cp-Ct-Cq surfaces completed at 50 %
Cp-Ct-Cq surfaces completed at 55 %
Cp-Ct-Cq surfaces completed at 60 %
Cp-Ct-Cq surfaces completed at 65 %
Cp-Ct-Cq surfaces completed at 70 %
Cp-Ct-Cq surfaces completed at 75 %
Cp-Ct-Cq surfaces completed at 80 %
Cp-Ct-Cq surfaces completed at 85 %
Cp-Ct-Cq surfaces completed at 90 %
Cp-Ct-Cq surfaces completed at 95 %
Cp-Ct-Cq surfaces completed at 100 %
-----------------------------------------------------------------------------
   Tuning a reference wind turbine controller using NREL's ROSCO to

Accessing Draft4Validator.resolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization.

----------------
Design Variables
----------------
name  val  size  lower  upper  
----  ---  ----  -----  ----- 

-----------
Constraints
-----------
name  val  size  lower  upper  equals  
----  ---  ----  -----  -----  ------ 

----------
Objectives
----------
name  val  size  
----  ---  ---- 

Run time (): 47.16603064537048


In [ ]:
# Lets analyize the baseline design!!
baseline_turb = pd.read_csv(os.path.join(WORKSHOP_REPO, stageFolder, "outputs/stage-0-baseline.csv"), index_col=0)
# Open the CSV file shown above so explore the various variables output from WISDEM/WEIS

baseline_turb.index # there are 4748 variables!!

Index(['sse_tune.v_min', 'sse_tune.v_max', 'sse_tune.r', 'sse_tune.chord',
       'sse_tune.theta', 'sse_tune.Rhub', 'sse_tune.Rtip',
       'sse_tune.hub_height', 'sse_tune.precone', 'sse_tune.tilt',
       ...
       'rotorse.rs.tot_loads_gust.Pz_af',
       'rotorse.stall_check.no_stall_constraint',
       'rotorse.stall_check.stall_angle_along_span', 'xf.span_start',
       'xf.cl_interp_flaps', 'xf.cd_interp_flaps', 'xf.cm_interp_flaps',
       'xf.flap_angles', 'xf.Re_loc', 'xf.Ma_loc'],
      dtype='object', name='variables', length=4748)

As you can see from the listing the variables in the CSV files, WEIS outputs a lot of information about the wind turbine model/run.

Lets look at a few of them to help us understand the baseline design.
- rotorse.rp.powercurve.V --> Wind speed (m/s)
- rotorse.rp.powercurve.Cp_aero --> Power Coefficient
- rotorse.rp.powercurve.Ct_aero --> Thrust Coefficient
- rotorse.rp.powercurve.P --> Power (W)
- rotorse.rp.powercurve.T --> Thrust (N)
- rotorse.rp.powercurve.pitch --> Pitch Angle (deg)